In [1]:
import moldesign as mdt
import moldesign.units as u

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
#mdt.configure()
u.

In [3]:
molecule = mdt.from_pdb('1yu8')

In [4]:
molecule
#molecule = mdt.Molecule([res for res in molecule.residues if res.type == 'protein'])

### Molecule: "1yu8" (600 atoms)

<a href="http://www.rcsb.org/pdb/explore.do?structureId=1yu8" target="_blank">HIGH-RESOLUTION CRYSTAL STRUCTURES OF VILLIN HEADPIECE AND MUTANTS WITH REDUCED F-ACTIN BINDING ACTIVITY.</a>

**Mass**: 8150.00 amu

**Formula**: C<sub>325</sub>N<sub>85</sub>O<sub>189</sub>S<sub>1</sub>

**Charge**: 0.0 elementary_charge

### Residues

| chain | protein | dna | rna | unknown | water | solvent |
|-|-|-|-|-|-|-|-|
| <b>X</b> | 64 |  |  |  | 92 |  ||

### Biopolymer chains

**X**: <span style="word-wrap:break-word;font-family:monospace"><span title="PRO10 (MISSING), chain X" style="color:Red">P</span><span title="THR11 (MISSING), chain X" style="color:Red">T</span><span title="LYS12 (MISSING), chain X" style="color:Red">K</span><span title="LEU13 (idx 0), chain X">L</span><span title="GLU14 (idx 1), chain X">E</span><span title="THR15 (idx 2), chain X">T</span><span title="PHE16 (idx 3), chain X">F</span><span title="PRO17 (idx 4), chain X">P</span><span title="LEU18 (idx 5), chain X">L</span><span title="ASP19 (idx 6), chain X">D</span><span title="VAL20 (idx 7), chain X">V</span><span title="LEU21 (idx 8), chain X">L</span><span title="VAL22 (idx 9), chain X">V</span><span title="ASN23 (idx 10), chain X">N</span><span title="THR24 (idx 11), chain X">T</span><span title="ALA25 (idx 12), chain X">A</span><span title="ALA26 (idx 13), chain X">A</span><span title="GLU27 (idx 14), chain X">E</span><span title="ASP28 (idx 15), chain X">D</span><span title="LEU29 (idx 16), chain X">L</span><span title="PRO30 (idx 17), chain X">P</span><span title="ARG31 (idx 18), chain X">R</span><span title="GLY32 (idx 19), chain X">G</span><span title="VAL33 (idx 20), chain X">V</span><span title="ASP34 (idx 21), chain X">D</span><span title="PRO35 (idx 22), chain X">P</span><span title="SER36 (idx 23), chain X">S</span><span title="ALA37 (idx 24), chain X">A</span><span title="LYS38 (idx 25), chain X">K</span><span title="GLU39 (idx 26), chain X">E</span><span title="ASN40 (idx 27), chain X">N</span><span title="HIS41 (idx 28), chain X">H</span><span title="LEU42 (idx 29), chain X">L</span><span title="SER43 (idx 30), chain X">S</span><span title="ASP44 (idx 31), chain X">D</span><span title="GLU45 (idx 32), chain X">E</span><span title="ASP46 (idx 33), chain X">D</span><span title="PHE47 (idx 34), chain X">F</span><span title="LYS48 (idx 35), chain X">K</span><span title="ALA49 (idx 36), chain X">A</span><span title="VAL50 (idx 37), chain X">V</span><span title="PHE51 (idx 38), chain X">F</span><span title="GLY52 (idx 39), chain X">G</span><span title="MET53 (idx 40), chain X">M</span><span title="THR54 (idx 41), chain X">T</span><span title="ARG55 (idx 42), chain X">R</span><span title="SER56 (idx 43), chain X">S</span><span title="ALA57 (idx 44), chain X">A</span><span title="PHE58 (idx 45), chain X">F</span><span title="ALA59 (idx 46), chain X">A</span><span title="ASN60 (idx 47), chain X">N</span><span title="LEU61 (idx 48), chain X">L</span><span title="PRO62 (idx 49), chain X">P</span><span title="LEU63 (idx 50), chain X">L</span><span title="TRP64 (idx 51), chain X">W</span><span title="LYS65 (idx 52), chain X">K</span><span title="GLN66 (idx 53), chain X">Q</span><span title="GLN67 (idx 54), chain X">Q</span><span title="ASN68 (idx 55), chain X">N</span><span title="LEU69 (idx 56), chain X">L</span><span title="LYS70 (idx 57), chain X">K</span><span title="LYS71 (idx 58), chain X">K</span><span title="GLU72 (idx 59), chain X">E</span><span title="LYS73 (idx 60), chain X">K</span><span title="GLY74 (idx 61), chain X">G</span><span title="LEU75 (idx 62), chain X">L</span><span title="PHE76 (idx 63), chain X">F</span></span>

In [5]:
molecule.draw()
#len(molecule.atoms)

In [6]:
import parmed as med

In [7]:
mdmol = mdt.assign_forcefield(molecule)

Connecting to docker host at unix://var/run/docker.sock... done


In [8]:
mdmol.ff.amber_params.prmtop.put('/tmp/tmp.prmtop')
parmedmol = med.load_file('/tmp/tmp.prmtop')

In [9]:
# mdmol.draw()
atom = mdmol.atoms[0]
atom.index

0

In [10]:
print atom.position[0]
print atom.position.value_in(u.angstrom)

10.367 ang
[ 10.367  29.903  18.753]


In [12]:
import tempfile
import shutil
import os

In [13]:


# In order to simluate using LAMMPS, we need:
#   get molecule from pdb, cif, or name
#   assign forcefields
#   load parmed structure


# LAMMPS UNITS:
# For style real, these are the units:

# mass = grams/mole
# distance = Angstroms
# time = femtoseconds
# energy = Kcal/mole
# velocity = Angstroms/femtosecond
# force = Kcal/mole-Angstrom
# torque = Kcal/mole
# temperature = Kelvin
# pressure = atmospheres
# dynamic viscosity = Poise
# charge = multiple of electron charge (1.0 is a proton)
# dipole = charge*Angstroms
# electric field = volts/Angstrom
# density = gram/cm^dim

# See http://parmed.github.io/ParmEd/html/index.html for ParmEd units

def formatLammpsData(lammps_data):
    predictable_filename = 'data.lammps_molecule'
    path = os.path.join(tmpdir, predictable_filename)

    with open(path, "w") as lammps_data:

        # print description
        lammps_data.write("LAMMPS Description\r\n\r\n")
        lammps_data.write("{0} atoms\r\n" .format(len(parmedmol.atoms)))
        lammps_data.write("{0} bonds\r\n" .format(len(parmedmol.bonds)))
        lammps_data.write("{0} angles\r\n" .format(len(parmedmol.angles)))
        lammps_data.write("{0} dihedrals\r\n" .format(len(parmedmol.dihedrals)))
        lammps_data.write("{0} impropers\r\n" .format(len(parmedmol.impropers)))

        lammps_data.write("\r\n")

        max_idx = 0
        for nonbond_idx in parmedmol.LJ_types.itervalues():
            if(nonbond_idx > max_idx):
                max_idx = nonbond_idx

        lammps_data.write("{0} atom types\r\n" .format(max_idx))
        lammps_data.write("{0} bond types\r\n" .format(len(parmedmol.bond_types)))
        lammps_data.write("{0} angle types\r\n" .format(len(parmedmol.angle_types)))
        lammps_data.write("{0} dihedral types\r\n" .format(len(parmedmol.dihedral_types)))
        lammps_data.write("{0} improper types\r\n" .format(len(parmedmol.improper_types)))

        lammps_data.write("\r\n")

        # box size
        # calculate lo and hi points
        xlo = xhi = ylo = yhi = zlo = zhi = None
        for atom in mdmol.atoms:
            if xlo == None:
                xlo = atom.x.value_in(u.angstrom)
                xhi = atom.x.value_in(u.angstrom)
                ylo = atom.y.value_in(u.angstrom)
                yhi = atom.y.value_in(u.angstrom)
                zlo = atom.z.value_in(u.angstrom)
                zhi = atom.z.value_in(u.angstrom)
            else:
                xlo = min(xlo, atom.x.value_in(u.angstrom))
                xhi = max(xhi, atom.x.value_in(u.angstrom))
                ylo = min(ylo, atom.y.value_in(u.angstrom))
                yhi = max(yhi, atom.y.value_in(u.angstrom))
                zlo = min(zlo, atom.z.value_in(u.angstrom))
                zhi = max(zhi, atom.z.value_in(u.angstrom))

        lammps_data.write("{0} {1} xlo xhi\r\n" .format(xlo, xhi))
        lammps_data.write("{0} {1} ylo yhi\r\n" .format(ylo, yhi))
        lammps_data.write("{0} {1} zlo zhi\r\n" .format(zlo, zhi))

        lammps_data.write("\r\n")

        # print masses -- Number of atom types
        lammps_data.write("Masses\r\n\r\n")
        for i in range(1, max_idx+1):
            for atom in parmedmol.atoms:
                if atom.nb_idx == i:
                    lammps_data.write("{0} {1}\r\n" .format(i, atom.mass))
                    break

        lammps_data.write("\r\n")


        # Pair Coeffs -- Number of atom types
        lammps_data.write("Pair Coeffs\r\n\r\n")
        for i in range(1, max_idx+1):
            for atom in parmedmol.atoms:
                if atom.nb_idx == i:
                    lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(i, atom.epsilon, atom.sigma, 
                                                                        atom.epsilon_14, atom.sigma_14))
                    break

        lammps_data.write("\r\n")


        # Bond Coeffs - Harmonic
        if parmedmol.bond_types:
            lammps_data.write("Bond Coeffs\r\n\r\n")
            for bt in parmedmol.bond_types:
                lammps_data.write("{0} {1} {2}\r\n" .format(bt.idx+1, bt.k, bt.req))

            lammps_data.write("\r\n")


        # Angle Coeffs - Harmonic
        if parmedmol.angle_types:
            lammps_data.write("Angle Coeffs\r\n\r\n")
            for at in parmedmol.angle_types:
                lammps_data.write("{0} {1} {2}\r\n" .format(at.idx+1, at.k, at.theteq))

            lammps_data.write("\r\n")


        # Dihedral Coeffs - Charmm
        if parmedmol.dihedral_types:
            lammps_data.write("Dihedral Coeffs\r\n\r\n")
            for dt in parmedmol.dihedral_types:
                lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(dt.idx+1, dt.phi_k, dt.per, 
                    (180 if dt.phase >= 180 else 0), 
                    (1.0 if dt.scnb == 1.0 else 0.0)))

            lammps_data.write("\r\n")


        # Improper Coeffs - Harmonic
        if parmedmol.improper_types:
            lammps_data.write("Improper Coeffs\r\n\r\n")
            for it in parmedmol.improper_types:
                print
                lammps_data.write("{0} {1} {2}\r\n" .format(it.idx+1, it.psi_k, it.psi_eq))

            lammps_data.write("\r\n")


        # print atoms
        lammps_data.write("Atoms\r\n\r\n")
        for atom in parmedmol.atoms:
            lammps_data.write("{0} {1} {2} {3} " .format(atom.idx+1, atom.residue.idx, atom.nb_idx, atom.charge))
            if not mdmol.positions.any():        
                lammps_data.write("{0} {1} {2} " .format(0.00, 0.00, 0.00))        
            else:
                pos = mdmol.positions[atom.idx]
                lammps_data.write("{0} {1} {2} " .format(pos[0].value_in(u.angstrom), pos[1].value_in(u.angstrom), 
                    pos[2].value_in(u.angstrom)))
            lammps_data.write("{0} {1} {2}\r\n" .format(0, 0, 0)) # nx, ny, nz not defined    

        lammps_data.write("\r\n")


        # print velocities
        lammps_data.write("Velocities\r\n\r\n")
        for atom in mdmol.atoms:
            xvel = atom.vx.value_in(u.angstrom/u.fs)
            yvel = atom.vy.value_in(u.angstrom/u.fs)
            zvel = atom.vz.value_in(u.angstrom/u.fs)
            lammps_data.write("{0} {1} {2} {3}\r\n" .format(atom.index+1, xvel, yvel, zvel))

        lammps_data.write("\r\n")


        # print bonds
        lammps_data.write("Bonds\r\n\r\n")
        for idx, bond in enumerate(parmedmol.bonds):
            lammps_data.write("{0} {1} {2} {3}\r\n" .format(idx+1, bond.type.idx+1, 
              bond.atom1.idx+1, bond.atom2.idx+1))

        lammps_data.write("\r\n")


        # print angles
        lammps_data.write("Angles\r\n\r\n")
        for idx, angle in enumerate(parmedmol.angles):
            lammps_data.write("{0} {1} {2} {3} {4}\r\n" .format(idx+1, angle.type.idx+1,
              angle.atom1.idx+1, 
              angle.atom2.idx+1, angle.atom3.idx+1))

        lammps_data.write("\r\n")


        # print dihedrals
        if parmedmol.dihedrals:
            lammps_data.write("Dihedrals\r\n\r\n")
            for idx, di in enumerate(parmedmol.dihedrals):
                lammps_data.write("{0} {1} {2} {3} {4} {5}\r\n" .format(idx+1, di.type.idx+1, 
                  di.atom1.idx+1, di.atom2.idx+1, 
                  di.atom3.idx+1, di.atom4.idx+1))

            lammps_data.write("\r\n")


        # print impropers
        if parmedmol.impropers:
            lammps_data.write("Impropers\r\n\r\n")
            for im in parmedmol.impropers:
                lammps_data.write("{0} {1} {2} {3} {4} {5}\r\n" .format(im.idx+1, im.type.idx+1, 
                  im.atom1.idx+1, im.atom2.idx+1, 
                  im.atom3.idx+1, im.atom4.idx+1))

            lammps_data.write("\r\n")

    
    #lammps_data.close()
    return path

In [14]:
tmpdir = tempfile.mkdtemp()

saved_umask = os.umask(0077)

path = formatLammpsData(tmpdir)
print path

/var/folders/l9/th_rqp090gg3375t46j58vsm0000gq/T/tmpwolwJo/data.lammps_molecule


In [15]:
def prepareLammps(dataPath):
    L.command("units real")
    L.command("atom_style full")
    L.command("pair_style lj/charmm/coul/long 8.0 10.0 10.0")
    L.command("bond_style harmonic")
    L.command("angle_style harmonic")
    L.command("dihedral_style charmm")
    L.command("improper_style harmonic")
    L.command("kspace_style pppm 0.0001")
    L.command("read_data " + dataPath)

def calculateEnergy(timeStep, outputFreq):
    #L.command("timestep " + str(round(timeStep,1)))
    L.command("thermo_style custom step temp pe etotal")
    #L.command("thermo " + str(outputFreq))

            
def groupHydrogens():
    groupCommand = "group hbonds type"
    for nonbond_name, nonbond_idx in parmedmol.LJ_types.iteritems():
        if(nonbond_name[0] == 'H'):
            groupCommand = groupCommand + " " + str(nonbond_idx)
    L.command(groupCommand)
    
    
def runNvt(timeStep, temp, damp, outputFreq, applyShake):
    L.command("neighbor 2.0 bin")
    #L.command("neigh_modify delay 5")
    L.command("timestep " + str(round(timeStep,1)))
    L.command("thermo_style custom step temp pe etotal")
    L.command("thermo " + str(outputFreq))
    nvt_command = "fix 1 all nvt temp {0} {1} {2}" .format(round(temp,1), round(temp,1), round(damp, 3))
    L.command(nvt_command)
    if(applyShake):
        groupHydrogens()
        shake_command = "fix 2 hbonds rattle 0.0001 20 0 t 5 6 m 1.0 a 31"
        L.command(shake_command)


In [16]:
from lammps import lammps, PyLammps

In [17]:
lmp = lammps()
L = PyLammps(ptr=lmp)

LAMMPS output is captured by PyLammps wrapper


In [18]:
prepareLammps(path)
# securely remove path
os.remove(path)
os.umask(saved_umask)
os.rmdir(tmpdir)

In [19]:
print "Before Evaluation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

Before Evaluation
(8.877, 11.103, 14.169)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)


In [20]:
calculateEnergy(1.0, 10)
L.run(0)
print L.eval("pe")
print L.atoms[1004].velocity




-4179.44402414
(0.0, 0.0, 0.0)


In [ ]:
if L.atoms[]
# copy molecule's positions and velocities to LAMMPS object
for atom in mdmol.atoms:
    #print atom.index
    L.atoms[atom.index].position = atom.position.value_in(u.angstrom)
    L.set("atom", atom.index, "vx", atom.velocity[0].value_in(u.angstrom / u.femtoseconds))
    L.set("atom", atom.index, "vy", atom.velocity[1].value_in(u.angstrom / u.femtoseconds))
    L.set("atom", atom.index, "vz", atom.velocity[2].value_in(u.angstrom / u.femtoseconds))

In [ ]:
print L.atoms.natoms

In [ ]:
L.atoms[1288].position

In [ ]:
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

In [ ]:
calculateEnergy(1.0, 10)
L.run(0)
print L.eval("pe")
print L.atoms[1004].velocity

In [ ]:
del L
del lmp

lmp = lammps()
L = PyLammps(ptr=lmp)

In [ ]:
prepareLammps(name)

In [ ]:
print "Before Simulation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

In [ ]:
runNvt(1.0, 300, 100, 10, True)


In [ ]:
# shake_command = "fix 2 hbonds shake 0.0001 10 0 b 4 6 8 10 12 14 18 a 31"
# L.command(shake_command)


In [ ]:
L.run(200)

In [ ]:
print "After Simulation"
print L.atoms[1012].position
print L.atoms[1012].velocity
print L.atoms[1012].force

In [ ]:
#

In [ ]:
# mdmol.draw3d() 17

In [ ]:
# for j in range(0, 10):
#     L.run(200)
#     print "After Simulation"
#     print L.atoms[1012].position
#     print L.atoms[1012].velocity
#     print L.atoms[1012].force
    

In [ ]:
L.run(10)

In [ ]:
for j in range(0, 10):
    L.run(1)
    print "After Simulation"
    print L.atoms[1012].position
    print L.atoms[1012].velocity
    print L.atoms[1012].force

In [ ]:
print mdmol.atoms[1004].position.value_in(u.angstrom)

In [ ]:
print L.atoms[1004].position

In [ ]:
L.atoms[1004].position = mdmol.atoms[1004].position.value_in(u.angstrom)
print L.atoms[1004].position

In [ ]:
print L.atoms[1004].velocity
print mdmol.atoms[1004].velocity.value_in(u.angstrom / u.femtoseconds)

In [ ]:
L.atoms[1004].velocity = [0.11, 0.22, 0.33]
print L.atoms[1004].velocity